In [297]:
%load_ext autoreload
%autoreload 2
%cd C:\MAD4AG
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
C:\MAD4AG


In [298]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm



import warnings

warnings.filterwarnings('ignore')

In [299]:
df_MAD = pd.read_parquet(f'./dbs/intermediate/indi_weights.parquet')


In [300]:
df_survey_twins = pd.read_pickle(f'./dbs/intermediate/df_survey_twins.pkl')


# add region information

In [301]:
lan= pd.read_csv(f'./dbs/lan.csv', delimiter=';', dtype={'code':'string'})
lan.rename(columns={'code':'county', 'landsdelar':'region'},inplace=True)

df_MAD = pd.merge(df_MAD, lan[['county','region']], on='county')
df_survey_twins = pd.merge(df_survey_twins, lan[['county','region']], on='county')


# calculate median distance

In [302]:
avg_dist_threshold = df_survey_twins.avg_dist.median()
com_dist_threshold = df_survey_twins.com_distance.median()

#df_survey_twins.avg_dist.quantile(0.5)


# create stratum

In [303]:
df_MAD['region'] = df_MAD['region'].replace({'Svealand':'0','Göteland':'1', 'Norrland':'2'})

df_MAD['urban_density'] = df_MAD['urban_density'].replace({'A': '0', 'B': '1', 'C': '1'})

df_MAD['avg_dist_cat'] = '0'
df_MAD['avg_dist_cat'][df_MAD.avg_dist > avg_dist_threshold] = '1'

df_MAD['com_dist_cat'] = '0'
df_MAD['com_dist_cat'][df_MAD.com_dist > com_dist_threshold] = '1'

In [304]:

df_MAD['stratum'] = df_MAD['region'].astype(str) + df_MAD['urban_density'].astype(str) +  df_MAD['commute'].astype(str)+  df_MAD['avg_dist_cat'].astype(str)

In [305]:

df_MAD['stratum'][(df_MAD.commute == 1)] = df_MAD['stratum'][(df_MAD.commute == 1)] + df_MAD['com_dist_cat'][(df_MAD.commute == 1)].astype(str)

#df_MAD['stratum'].value_counts()

In [306]:
df_survey_twins['region'] = df_survey_twins['region'].replace({'Svealand':'0','Göteland':'1', 'Norrland':'2'})

df_survey_twins['urban_density'] = df_survey_twins['urban_density'].replace({'A': '0', 'B': '1', 'C': '1'})

df_survey_twins['avg_dist_cat'] = '0'
df_survey_twins['avg_dist_cat'][df_survey_twins.avg_dist > avg_dist_threshold] = '1'

df_survey_twins['com_dist_cat'] = '0'
df_survey_twins['com_dist_cat'][df_survey_twins.com_distance > com_dist_threshold] = '1'


In [307]:

df_survey_twins['stratum'] = df_survey_twins['region'].astype(str) + df_survey_twins['urban_density'].astype(str) +  df_survey_twins['commute'].astype(str)+  df_survey_twins['avg_dist_cat'].astype(str)

In [308]:

df_survey_twins['stratum'][(df_survey_twins.commute == 1)] = df_survey_twins['stratum'][(df_survey_twins.commute == 1)] + df_survey_twins['com_dist_cat'][(df_survey_twins.commute == 1)].astype(str)

df_survey_twins['stratum'].value_counts()

1100     2588
11100    1935
11111    1913
1101     1552
0100     1108
01100     853
01111     807
0101      614
1001      579
10111     546
11110     410
2100      241
21100     234
0001      174
01110     165
00111     155
11101     149
1000      125
2001      124
2101       95
10100      91
21111      90
20111      83
10110      82
01101      72
21110      29
00110      17
0000       14
00100      10
21101       9
10101       7
20110       3
20100       2
2000        1
Name: stratum, dtype: int64

In [309]:
list_survey_type = list(df_survey_twins['stratum'].unique())
list_MAD_type = list(df_MAD['stratum'].unique())


set(list_MAD_type).difference(set(list_survey_type))

{'00101', '20101'}

In [310]:
threshold_count = 50

df_stratum = pd.DataFrame(columns=['stratum','level','mad_size', 'sur_size','survey_ppl'])

In [311]:
#list_MAD_type= ['1100','1101']

for i in list_MAD_type:
    df_MAD_p_type = df_MAD[df_MAD['stratum']==i]
    #print('mad len: ', len(df_MAD_p_type))

    df_sur_p_type = df_survey_twins[df_survey_twins['region']==i[0]]

    df_sur_temp = df_sur_p_type
    level = 0
    if len(df_sur_p_type) > threshold_count:
        df_sur_temp = df_sur_p_type[df_sur_p_type['urban_density']==i[1]]
        level = 1
        if len(df_sur_temp) > threshold_count:
            df_sur_p_type = df_sur_temp
            df_sur_temp = df_sur_p_type[df_sur_p_type['commute']==int(i[2])]
            level = 2
            #print(level)
            if len(df_sur_temp) > threshold_count:
                df_sur_p_type = df_sur_temp
                df_sur_temp = df_sur_p_type[df_sur_p_type['avg_dist_cat']==i[3]]
                level = 3
                #print(int(i[2]))
                if (len(df_sur_temp) > threshold_count)&(int(i[2])==0):
                   df_sur_p_type = df_sur_temp
                   level = 4

                elif (len(df_sur_temp) > threshold_count)&(int(i[2])==1):
                    df_sur_p_type = df_sur_temp
                    df_sur_temp = df_sur_p_type[df_sur_p_type['com_dist_cat']==i[4]]
                    level = 4
                    #print(level)

                    if len(df_sur_temp) > threshold_count:
                        df_sur_p_type = df_sur_temp
                        level = 5

    df_stratum = df_stratum.append(dict(stratum=i, level=level, mad_size=len(df_MAD_p_type), sur_size=len(df_sur_p_type), survey_ppl=df_sur_p_type['sub_id'].tolist()), ignore_index=True)




In [312]:
from collections import Counter



def unique_activities(data):
    act_tuple = data['act_seq']
    #data['act_uniq'] = tuple(set(Counter(act_tuple).keys()))
    data['act_uniq'] = tuple(Counter(act_tuple).keys())

    # this will take only unique numbers from the tuple
    return data

tqdm.pandas()
df_survey_twins = df_survey_twins.apply(lambda row: unique_activities(row), axis=1)

In [313]:
seq_dist = df_survey_twins.act_uniq.value_counts(normalize=True).reset_index(name='count_desired')
seq_dist['count_desired']=seq_dist['count_desired']*len(df_MAD)

In [314]:
df_candidates = pd.DataFrame(columns=['sub_id', 'act_seq', 'act_uniq','stratum','mad_size' ,'ratio'])


for index, row in df_stratum.iterrows():

    df_temp= df_survey_twins[['sub_id', 'act_seq', 'act_uniq']][df_survey_twins.sub_id.isin(row['survey_ppl'])]
    df_temp['stratum']= row['stratum']
    df_temp['mad_size']=row['mad_size']
    df_temp['ratio']=row['mad_size']/row['sur_size']
    df_candidates = df_candidates.append(df_temp)
    #print(row['survey_ppl'])
    #len(df_survey_twins[df_survey_twins.sub_id.isin(row['survey_ppl'])])



In [315]:
df_candidates.ratio.sum()

258666.99999999994

In [316]:
df_candidates.groupby('act_uniq')['ratio'].sum()

act_uniq
(Home, Home, Home, Home, Home, Other, Other, Other, Home)                      10.918826
(Home, Home, Home, Home, Other, Home, Home)                                   113.854710
(Home, Home, Home, Home, Other, Home, Other, Home, Home)                       25.720000
(Home, Home, Home, Home, Other, Other, Home)                                   10.918826
(Home, Home, Home, Home, Work)                                                  6.028941
                                                                                 ...    
(Work, Other, Other, Other, Other, Other, Other, Other, Home, Other, Home)      4.890110
(Work, Other, Work, Home)                                                       6.028941
(Work, Other, Work, Other, Home)                                               37.012757
(Work, Other, Work, Other, Home, Other, Home, Other, Home)                     11.674102
(Work, Work, Home)                                                             25.338655
Name: ratio,

In [317]:
for i in range(20):

    #fit to candidates ratios to the act seq distribution
    bbb = df_candidates.groupby('act_uniq')['ratio'].sum().reset_index(name='count')
    merged_df = pd.merge(seq_dist, bbb, left_on='index', right_on='act_uniq', how='inner')
    merged_df['result_column'] = merged_df['count_desired'] / merged_df['count']
    df_candidates = pd.merge(df_candidates, merged_df[['act_uniq', 'result_column']], on='act_uniq')

    df_candidates['ratio']= df_candidates['ratio']*df_candidates['result_column']
    df_candidates.drop(columns='result_column', inplace=True)

    #fit to candidates ratios to the mad size by stratum
    bbb = df_candidates.groupby('stratum')['ratio'].sum().reset_index(name='count')
    df_candidates = pd.merge(df_candidates, bbb, on='stratum')

    df_candidates['ratio'] = df_candidates['ratio'] * (df_candidates['mad_size'] / df_candidates['count'])
    df_candidates.drop(columns='count', inplace=True)





In [318]:
df_candidates.groupby('act_uniq')['ratio'].sum()


act_uniq
(Home, Home, Home, Home, Home, Other, Other, Other, Home)                     26.740715
(Home, Home, Home, Home, Other, Home, Home)                                   106.96286
(Home, Home, Home, Home, Other, Home, Other, Home, Home)                      26.740715
(Home, Home, Home, Home, Other, Other, Home)                                  26.740715
(Home, Home, Home, Home, Work)                                                 8.576687
                                                                                ...    
(Work, Other, Other, Other, Other, Other, Other, Other, Home, Other, Home)     8.576687
(Work, Other, Work, Home)                                                      8.576687
(Work, Other, Work, Other, Home)                                              17.153374
(Work, Other, Work, Other, Home, Other, Home, Other, Home)                     8.576687
(Work, Work, Home)                                                             8.576687
Name: ratio, Length: 92

In [319]:
df_candidates['probability'] = df_candidates['ratio']/df_candidates['mad_size']

df_MAD['twin_ppl'] =0

In [320]:
df_candidates_prob= df_candidates.groupby('stratum')['probability'].apply(list).reset_index(name='probability_list')


df_candidates_ppl= df_candidates.groupby('stratum')['sub_id'].apply(list).reset_index(name='ppl_list')

In [321]:
df_stratum = pd.merge(df_stratum, df_candidates_prob)

df_stratum = pd.merge(df_stratum, df_candidates_ppl)


In [322]:
df_stratum['twin_ppl'] = df_stratum.apply( lambda row: np.argmax(np.random.multinomial(1, row['probability_list'], row['mad_size']), axis=1).tolist()  , axis=1)

In [323]:
# len(df_stratum.loc[20,'twin_ppl'])
# twin_ppl_list = df_stratum.loc[20,'twin_ppl']
# twin_ppl_list = Counter(twin_ppl_list)


In [324]:
df_survey_twins_new = df_survey_twins.set_index('sub_id')
df_survey_twins_new = df_survey_twins_new[['act_seq', 'act_uniq', 'avg_dist']]


df_twins = pd.DataFrame()

for index, row in df_stratum.iterrows():
    twin_ppl_list = df_stratum.loc[index,'ppl_list']
    twin_ppl_place = df_stratum.loc[index,'twin_ppl']
    twin_ppl_list = [twin_ppl_list[place] for place in twin_ppl_place]

    df_survey_twins_new_candidates= df_survey_twins_new.loc[twin_ppl_list].reset_index()

    df_survey_twins_new_candidates.sort_values(by=['avg_dist'], inplace=True)
    df_survey_twins_new_candidates.reset_index(inplace=True)
    df_survey_twins_new_candidates.drop(columns='index', inplace=True)

    df_MAD_new = df_MAD[df_MAD.stratum==row['stratum']]
    df_MAD_new.sort_values(by=['avg_dist'], inplace=True)
    df_MAD_new.reset_index(inplace=True)
    df_MAD_new = pd.concat([df_MAD_new, df_survey_twins_new_candidates], axis=1, ignore_index=False)
    df_twins = df_twins.append(df_MAD_new)

    print(index)





0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35


In [325]:
print('The share of the matched survey ppl: ', np.round(len(df_twins.sub_id.unique())/len(df_survey_twins.sub_id.unique()),2))

The share of the matched survey ppl:  1.0


In [326]:
df_twins.sub_id.value_counts()

20122331011    127
20111431040    115
20111531054    115
20123211028    113
20110521076    109
              ... 
20143231048      1
20113411061      1
20111541002      1
20140411045      1
20113231058      1
Name: sub_id, Length: 14840, dtype: int64

In [327]:
df_twins.act_uniq.value_counts(normalize=True)


(Home, Other, Home)                                                  0.287791
(Home, Work, Home)                                                   0.094597
(Home, Other, Home, Other, Home)                                     0.078993
(Home, Other, Other, Home)                                           0.076353
(Home, Other, Other, Other, Home)                                    0.029876
                                                                       ...   
(Work, Work, Home)                                                   0.000008
(Home, Other, Home, Other, Work, Other, Work, Other, Home)           0.000008
(Home, Home, Work, Home, Other, Other, Other, Home, Home)            0.000004
(Home, Other, Work, Other, Other, Work, Home, Other, Other, Home)    0.000004
(Home, Work, Other, Other, Work, Other, Other, Other, Work, Home)    0.000004
Name: act_uniq, Length: 927, dtype: float64

In [328]:
df_survey_twins.act_uniq.value_counts(normalize=True)


(Home, Work, Home)                                                              0.192982
(Home, Other, Home)                                                             0.186933
(Home, Other, Home, Other, Home)                                                0.051691
(Home, Other, Other, Home)                                                      0.050212
(Home, Work, Other, Home)                                                       0.045910
                                                                                  ...   
(Home, Other, Home, Other, Work, Home, Other, Home)                             0.000067
(Home, Other, School, Home, Work, Other, Home)                                  0.000067
(Home, Other, Home, Work, Other, Work, Other, Home)                             0.000067
(Home, Other, Other, School, School, Home)                                      0.000067
(Home, Other, Home, Home, Work, Home, Other, Home, Other, Home, Other, Home)    0.000067
Name: act_uniq, Lengt

In [329]:
df_twins.act_seq.value_counts(normalize=True)


(Home, Other, Home)                                                  0.287791
(Home, Work, Home)                                                   0.094597
(Home, Other, Home, Other, Home)                                     0.078993
(Home, Other, Other, Home)                                           0.076353
(Home, Other, Other, Other, Home)                                    0.029876
                                                                       ...   
(Work, Work, Home)                                                   0.000008
(Home, Other, Home, Other, Work, Other, Work, Other, Home)           0.000008
(Home, Home, Work, Home, Other, Other, Other, Home, Home)            0.000004
(Home, Other, Work, Other, Other, Work, Home, Other, Other, Home)    0.000004
(Home, Work, Other, Other, Work, Other, Other, Other, Work, Home)    0.000004
Name: act_seq, Length: 927, dtype: float64

In [330]:
df_survey_twins.act_seq.value_counts(normalize=True)


(Home, Work, Home)                                                              0.192982
(Home, Other, Home)                                                             0.186933
(Home, Other, Home, Other, Home)                                                0.051691
(Home, Other, Other, Home)                                                      0.050212
(Home, Work, Other, Home)                                                       0.045910
                                                                                  ...   
(Home, Other, Home, Other, Work, Home, Other, Home)                             0.000067
(Home, Other, School, Home, Work, Other, Home)                                  0.000067
(Home, Other, Home, Work, Other, Work, Other, Home)                             0.000067
(Home, Other, Other, School, School, Home)                                      0.000067
(Home, Other, Home, Home, Work, Home, Other, Home, Other, Home, Other, Home)    0.000067
Name: act_seq, Length

In [331]:
df_candidates[df_candidates.sub_id==20113751034]

,sub_id,act_seq,act_uniq,stratum,mad_size,ratio,probability
15742,20113751034,"(Work, Home, Other, Home, Other, Other, Home, ...","(Work, Home, Other, Home, Other, Other, Home, ...",1101,34118,26.740715,0.000784


In [332]:
aaa= df_candidates[df_candidates.stratum=='1101']

In [333]:
df_survey_twins[df_survey_twins.sub_id==20113751034]


,sub_id,act_seq,act_uniq,origin_main_deso,avg_dist,commute,com_distance,county,urban_density,region,avg_dist_cat,com_dist_cat,stratum
5319,20113751034,"(Work, Home, Other, Home, Other, Other, Home, ...","(Work, Home, Other, Home, Other, Other, Home, ...",1442C1040,12575.112668,0,NaN,14,1,1,1,0,1101


In [334]:
aab= df_survey_twins[df_survey_twins.stratum=='1101']
len(aab[aab.avg_dist<avg_dist_threshold])

0

In [335]:
len(df_survey_twins[(df_survey_twins.region=='0')&(df_survey_twins.urban_density=='0')&(df_survey_twins.commute==1)&(df_survey_twins.avg_dist_cat=='0')])


10

In [336]:
len(df_survey_twins[(df_survey_twins.region=='0')&(df_survey_twins.urban_density=='0')&(df_survey_twins.commute==1)])

182